# CIFAR-10 Image Classification with CNN

This notebook demonstrates how to build and train a Convolutional Neural Network (CNN) to classify images from the CIFAR-10 dataset.

**CIFAR-10 Dataset:**
- 60,000 32x32 color images in 10 classes
- 50,000 training images and 10,000 test images
- Classes: airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck

## 1. Import Required Libraries

In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

TensorFlow version: 2.19.0
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## 2. Load and Explore the CIFAR-10 Dataset

In [ ]:
# Load the CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Define class names
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

# Print dataset information
print(f"Training images shape: {train_images.shape}")
print(f"Training labels shape: {train_labels.shape}")
print(f"Test images shape: {test_images.shape}")
print(f"Test labels shape: {test_labels.shape}")
print(f"\nNumber of training samples: {len(train_images)}")
print(f"Number of test samples: {len(test_images)}")
print(f"Image dimensions: {train_images.shape[1:]}")

## 3. Visualize Sample Images

In [ ]:
# Display the first 25 images from the training set
plt.figure(figsize=(12, 12))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i])
    plt.xlabel(class_names[train_labels[i][0]], fontsize=10)
plt.tight_layout()
plt.show()

## 4. Data Preprocessing

In [ ]:
# Normalize pixel values to be between 0 and 1
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

print("Data normalized successfully!")
print(f"Training data range: [{train_images.min()}, {train_images.max()}]")
print(f"Test data range: [{test_images.min()}, {test_images.max()}]")

## 5. Build the CNN Model

We'll create a Convolutional Neural Network with:
- 3 Convolutional layers with ReLU activation
- MaxPooling layers for downsampling
- Dropout for regularization
- Dense layers for classification

In [ ]:
def create_model():
    model = models.Sequential([
        # First Convolutional Block
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Second Convolutional Block
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Third Convolutional Block
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Dense Layers
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')  # 10 classes
    ])
    
    return model

# Create the model
model = create_model()

# Display model architecture
model.summary()

## 6. Compile the Model

In [ ]:
# Compile the model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("Model compiled successfully!")

## 7. Set Up Callbacks

In [ ]:
# Early stopping to prevent overfitting
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

# Reduce learning rate when validation loss plateaus
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-7,
    verbose=1
)

callbacks = [early_stopping, reduce_lr]

## 8. Train the Model

In [ ]:
# Train the model
history = model.fit(
    train_images, 
    train_labels,
    epochs=50,
    batch_size=64,
    validation_data=(test_images, test_labels),
    callbacks=callbacks,
    verbose=1
)

## 9. Visualize Training History

In [ ]:
# Plot training & validation accuracy and loss
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Accuracy plot
ax1.plot(history.history['accuracy'], label='Training Accuracy', linewidth=2)
ax1.plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2)
ax1.set_title('Model Accuracy', fontsize=14, fontweight='bold')
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Accuracy', fontsize=12)
ax1.legend(loc='lower right')
ax1.grid(True, alpha=0.3)

# Loss plot
ax2.plot(history.history['loss'], label='Training Loss', linewidth=2)
ax2.plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
ax2.set_title('Model Loss', fontsize=14, fontweight='bold')
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Loss', fontsize=12)
ax2.legend(loc='upper right')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 10. Evaluate the Model

In [ ]:
# Evaluate on test set
test_loss, test_accuracy = model.evaluate(test_images, test_labels, verbose=2)

print(f"\n{'='*50}")
print(f"Test Accuracy: {test_accuracy*100:.2f}%")
print(f"Test Loss: {test_loss:.4f}")
print(f"{'='*50}")

## 11. Make Predictions on Test Images

In [ ]:
# Make predictions on test set
predictions = model.predict(test_images)

# Function to display predictions
def plot_predictions(images, true_labels, predictions, class_names, num_images=15):
    plt.figure(figsize=(15, 10))
    for i in range(num_images):
        plt.subplot(3, 5, i + 1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(images[i])
        
        predicted_label = np.argmax(predictions[i])
        true_label = true_labels[i][0]
        confidence = 100 * np.max(predictions[i])
        
        # Color code: green for correct, red for incorrect
        color = 'green' if predicted_label == true_label else 'red'
        
        plt.xlabel(f"Pred: {class_names[predicted_label]}\nTrue: {class_names[true_label]}\nConf: {confidence:.1f}%",
                   color=color, fontsize=9)
    
    plt.tight_layout()
    plt.show()

# Display predictions
plot_predictions(test_images, test_labels, predictions, class_names)

## 12. Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Get predicted classes
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_labels.flatten()

# Create confusion matrix
cm = confusion_matrix(true_classes, predicted_classes)

# Plot confusion matrix
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names,
            cbar_kws={'label': 'Count'})
plt.title('Confusion Matrix', fontsize=16, fontweight='bold', pad=20)
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Print classification report
print("\nClassification Report:")
print("="*70)
print(classification_report(true_classes, predicted_classes, target_names=class_names))

## 13. Per-Class Accuracy

In [ ]:
# Calculate per-class accuracy
class_accuracy = cm.diagonal() / cm.sum(axis=1)

# Create bar plot
plt.figure(figsize=(12, 6))
bars = plt.bar(class_names, class_accuracy * 100, color='steelblue', edgecolor='black', linewidth=1.2)

# Color bars based on accuracy
for i, bar in enumerate(bars):
    if class_accuracy[i] >= 0.8:
        bar.set_color('green')
    elif class_accuracy[i] >= 0.6:
        bar.set_color('orange')
    else:
        bar.set_color('red')

plt.xlabel('Class', fontsize=12, fontweight='bold')
plt.ylabel('Accuracy (%)', fontsize=12, fontweight='bold')
plt.title('Per-Class Accuracy', fontsize=14, fontweight='bold', pad=15)
plt.xticks(rotation=45, ha='right')
plt.ylim(0, 100)
plt.grid(axis='y', alpha=0.3, linestyle='--')

# Add value labels on bars
for i, (name, acc) in enumerate(zip(class_names, class_accuracy)):
    plt.text(i, acc * 100 + 2, f'{acc*100:.1f}%', ha='center', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.show()

# Print per-class accuracy
print("\nPer-Class Accuracy:")
print("="*40)
for name, acc in zip(class_names, class_accuracy):
    print(f"{name:12s}: {acc*100:6.2f}%")

## 14. Save the Model

In [ ]:
# Save the model
model.save('cifar10_cnn_model.h5')
print("Model saved as 'cifar10_cnn_model.h5'")

# Also save in TensorFlow SavedModel format
model.save('cifar10_cnn_model')
print("Model saved as 'cifar10_cnn_model' (SavedModel format)")

## 15. Test with Custom Predictions

In [ ]:
# Function to predict a single image
def predict_single_image(model, image, true_label=None):
    # Add batch dimension
    img_array = np.expand_dims(image, axis=0)
    
    # Make prediction
    prediction = model.predict(img_array, verbose=0)
    predicted_class = np.argmax(prediction[0])
    confidence = np.max(prediction[0]) * 100
    
    # Display image and prediction
    plt.figure(figsize=(6, 6))
    plt.imshow(image)
    plt.axis('off')
    
    title = f"Predicted: {class_names[predicted_class]} ({confidence:.2f}%)"
    if true_label is not None:
        title += f"\nTrue: {class_names[true_label]}"
        color = 'green' if predicted_class == true_label else 'red'
    else:
        color = 'blue'
    
    plt.title(title, fontsize=14, fontweight='bold', color=color)
    plt.tight_layout()
    plt.show()
    
    # Print top 3 predictions
    top_3_indices = np.argsort(prediction[0])[-3:][::-1]
    print("\nTop 3 Predictions:")
    print("="*40)
    for i, idx in enumerate(top_3_indices, 1):
        print(f"{i}. {class_names[idx]:12s}: {prediction[0][idx]*100:6.2f}%")

# Test with a random image from test set
random_idx = np.random.randint(0, len(test_images))
predict_single_image(model, test_images[random_idx], test_labels[random_idx][0])

## Summary

This notebook demonstrated:
1. ✅ Loading and exploring the CIFAR-10 dataset
2. ✅ Data preprocessing and normalization
3. ✅ Building a CNN architecture with batch normalization and dropout
4. ✅ Training with callbacks (early stopping, learning rate reduction)
5. ✅ Visualizing training history
6. ✅ Evaluating model performance
7. ✅ Analyzing predictions with confusion matrix
8. ✅ Per-class accuracy analysis
9. ✅ Saving the trained model

**Expected Performance:** With this architecture, you should achieve around 75-85% accuracy on the test set.

**Next Steps:**
- Try data augmentation to improve accuracy
- Experiment with different architectures (ResNet, VGG, etc.)
- Use transfer learning with pre-trained models
- Implement learning rate scheduling
- Try ensemble methods